In [4]:
from sklearn.linear_model import SGDClassifier, RidgeClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC,LinearSVC
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, plot_confusion_matrix, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import os
import pickle
import pandas as pd
from sklearn.utils import resample

pd.options.display.max_rows = 35 
pd.options.display.max_columns = None

In [5]:
def get_pickles(): 
    x_train = pickle.load(open('../Pickles/x_train.p', 'rb'))
    x_test = pickle.load(open('../Pickles/x_test.p', 'rb'))
    y_train = pickle.load(open('../Pickles/y_train.p', 'rb'))
    y_test = pickle.load(open('../Pickles/y_test.p', 'rb'))
    return x_train, x_test, y_train, y_test


x_train, x_test, y_train, y_test = get_pickles()
print(x_train.shape, x_test.shape) 
print(y_train.shape, y_test.shape)

(6702, 5000) (2234, 5000)
(6702,) (2234,)


### Resampling
- 0 = negative
- 1 = neutral 
- 2 = positive

In [6]:
print('Original Train Value Counts')
print(y_train.value_counts())
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print('Original Test Value Counts')
print(y_test.value_counts())
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')


train_df = pd.DataFrame(x_train)
train_df['target'] = y_train.values 

neg_df = train_df[train_df.target == 0]
neut_df = train_df[train_df.target == 1]
pos_df = train_df[train_df.target == 2]



resample_pos = resample(pos_df, n_samples = 500, random_state = 10, replace = True)
resample_neut = resample(neut_df, n_samples = 500, random_state = 10, replace = True)

final_df= neg_df.append(resample_pos, ignore_index = True)
final_df = final_df.append(resample_neut, ignore_index = True)
print('Final Resampled Value Counts')
print(final_df.target.value_counts())
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')


x_train_new = final_df[[i for i in final_df.columns if i != 'target']]
y_train_new = final_df[['target']].values.ravel()

Original Train Value Counts
1.0    4041
2.0    2233
0.0     428
Name: Emotion_New, dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Original Test Value Counts
1.0    1347
2.0     745
0.0     142
Name: Emotion_New, dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Final Resampled Value Counts
1.0    500
2.0    500
0.0    428
Name: target, dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [ ]:
models = {'DT': DecisionTreeClassifier(), 'Gaussian': GaussianNB(), 'LDA': LinearDiscriminantAnalysis(),
           'LinearSVC': LinearSVC(max_iter = 1250), 'SDGSVC': SGDClassifier(),  
          'ADA': AdaBoostClassifier(), 'Bagging': BaggingClassifier(), 'Ridge': RidgeClassifier(), 
          'RF': RandomForestClassifier()}

#create stacked model
stack_m = [] 
for model, m in models.items(): 
    stack_m.append((model, m))
stack_model = StackingClassifier(estimators = stack_m, final_estimator = LogisticRegression(), cv = 5)
models['stacked'] = stack_model

#test each model and stacking
results = []
model_names = []
pbar = tqdm(models.items())
for model, m in pbar: 
    pbar.set_description(f'Evaluating {model.upper()}')
    cv = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 5)
    scores = cross_val_score(m, x_train_new, y_train_new, scoring = 'accuracy', cv = cv, n_jobs = 12, 
                             error_score = 'raise')
    results.append(scores)
    model_names.append(model)

Evaluating STACKED:  90%|█████████████████████████████████████████████████████████████████████████████████         | 9/10 [02:54<00:18, 18.16s/it]

In [ ]:
vanilla_dict = {i:y for i,y in zip(model_names, results)}
pickle.dump(vanilla_dict, open('models/VanillaResults.p', 'wb'))

In [ ]:
plt.figure(figsize = (10,8))
plt.boxplot(results, labels = model_names, showmeans = True)
plt.title('Accuracy for Each Vanilla Model')
plt.ylabel('Accuracy'); plt.xlabel('Model')
plt.savefig('figures/BaselineAccuracy.png')
plt.show()

In [ ]:
assert False

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state = 10, n_jobs = -1)

rf.fit(x_train_new, y_train_new)

train_pred = rf.predict(x_train_new)
test_pred = rf.predict(x_test)

print('Vanilla Random Forest')
print(f'Train Accuracy: {rf.score(x_train_new, y_train_new)}')
print(f'Test Accuracy: {rf.score(x_test, y_test)}')

### Bagging